In [1]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
#Load in the data set
FifaDB = pd.read_csv("NewFifa.csv")
FifaDB.set_index('Name', inplace=True)

In [7]:
#Remove Goalkeepers
FifaDB = FifaDB[FifaDB.Position != 5]
FifaDB.head(10)

,Overall,Potential,Value,Wage,Preferred Foot,Weak Foot,Skill Moves,Work Rate,Body Type,Position,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
Name,,,,,,,,,,,,,,,,,,,,,
L. Messi,94,94,110000000,565000,0,4.0,4.0,8,4,21,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
Cristiano Ronaldo,94,94,77000000,405000,1,4.0,5.0,1,1,26,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
Neymar Jr,92,93,118000000,290000,1,5.0,5.0,2,5,14,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
K. De Bruyne,91,92,102000000,355000,1,5.0,4.0,0,6,19,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0
E. Hazard,91,91,93000000,340000,1,4.0,4.0,2,6,11,...,86.0,91.0,34.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0
L. Modrić,91,91,67000000,420000,1,4.0,4.0,0,3,19,...,82.0,84.0,60.0,76.0,73.0,13.0,9.0,7.0,14.0,9.0
L. Suárez,91,91,80000000,455000,1,4.0,3.0,2,6,23,...,85.0,85.0,62.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0
Sergio Ramos,91,91,51000000,380000,1,3.0,3.0,2,6,18,...,75.0,82.0,87.0,92.0,91.0,11.0,8.0,9.0,7.0,11.0
R. Lewandowski,90,90,77000000,205000,1,4.0,4.0,2,6,26,...,88.0,86.0,34.0,42.0,19.0,15.0,6.0,12.0,8.0,10.0


In [4]:
# #Checking correlations for certain positions
# plt.figure(figsize=(12,10))
# cor = FifaDB.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# #Correlation with output variable
# cor_target = abs(cor["Position_CB"])
# #Selecting highly correlated features
# relevant_features = cor_target[cor_target>0.2]
# relevant_features

In [5]:
# #Random Forest
# print("The shape of our features is:", features.shape)